# AUTOCREATE システム総合テスト・ガイド

## 🎯 このノートブックについて

このJupyter Notebookは、AUTOCREATEプロジェクトの**動くドキュメント**として機能します。  
以下の機能を提供します：

- 📚 **説明とテストの一体化** - 各機能の説明とテストコードを同じ場所で管理
- 🔍 **Make コマンド実行ガイド** - 忘れがちなコマンドを実行可能な形で記録
- 🏥 **システムヘルスチェック** - 全体の動作確認をワンクリック
- 🛠️ **トラブルシューティング** - 問題解決手順を実行可能な形で記録

## 🏗️ プロジェクト構成

- **8つのGradioインターフェース** がFastAPIアプリに統合
- **Laravel風API基盤** でRESTful APIを提供
- **システム監視・ヘルスチェック** 機能
- **Docker環境** でのPCレス・ブラウザ操作対応

In [ ]:
# 必要なライブラリをインポート
import os
import sys
import subprocess
import requests
import json
import time
import psutil
from datetime import datetime
from pathlib import Path

# プロジェクトルートの設定
PROJECT_ROOT = "/workspaces/AUTOCREATE"
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print(f"📍 現在のディレクトリ: {os.getcwd()}")
print(f"🐍 Python バージョン: {sys.version}")
print(f"📅 実行日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("✅ 環境設定完了")

In [ ]:
# 🛠️ Make コマンド実行用ヘルパー関数

def run_make_command(command, capture_output=True, timeout=30):
    """
    Makeコマンドを実行し、結果を表示する
    
    Args:
        command (str): 実行するMakeコマンド（make の後の部分）
        capture_output (bool): 出力をキャプチャするかどうか
        timeout (int): タイムアウト時間（秒）
    """
    full_command = f"make {command}"
    print(f"🔨 実行コマンド: {full_command}")
    print("-" * 50)
    
    try:
        if capture_output:
            result = subprocess.run(
                full_command, 
                shell=True, 
                capture_output=True, 
                text=True, 
                timeout=timeout,
                cwd=PROJECT_ROOT
            )
            print("📤 標準出力:")
            print(result.stdout)
            if result.stderr:
                print("❌ エラー出力:")
                print(result.stderr)
            print(f"🔢 終了コード: {result.returncode}")
        else:
            # リアルタイム出力の場合
            process = subprocess.Popen(
                full_command,
                shell=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                cwd=PROJECT_ROOT
            )
            
            while True:
                output = process.stdout.readline()
                if output == '' and process.poll() is not None:
                    break
                if output:
                    print(output.strip())
    
    except subprocess.TimeoutExpired:
        print(f"⏰ タイムアウト: {timeout}秒で処理が完了しませんでした")
    except Exception as e:
        print(f"❌ エラーが発生しました: {str(e)}")

def check_service_health(url, service_name):
    """
    サービスのヘルスチェックを実行
    """
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            print(f"✅ {service_name}: 正常稼働中")
            return True
        else:
            print(f"⚠️ {service_name}: HTTPステータス {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"❌ {service_name}: 接続エラー - {str(e)}")
        return False

print("✅ ヘルパー関数の準備完了")

---

## 📋 1. Make コマンド一覧・実行ガイド

このセクションでは、プロジェクトで使用可能な全てのMakeコマンドを実行可能な形で記録します。
忘れがちなコマンドも、このノートブックから直接実行できます。

### 🔍 利用可能なコマンド確認

In [ ]:
# 利用可能なMakeコマンドの一覧を表示
run_make_command("help")

### 🚀 アプリケーション起動・停止コマンド

In [ ]:
# ✋ ポート7860で動作中のプロセスを停止
print("ポート7860で動作中のプロセスを停止します...")
run_make_command("stop-port")

In [ ]:
# 🚀 FastAPIアプリケーションを起動（バックグラウンド）
print("⚠️ 注意: このコマンドはバックグラウンドで起動します")
print("起動後は別のターミナルで 'make stop-port' を実行して停止してください")
print("")

# バックグラウンド起動の場合（実際には別の方法が推奨）
import threading

def start_app_background():
    run_make_command("app", capture_output=False, timeout=300)

# 起動状況を確認
print("現在のポート7860の使用状況:")
try:
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('localhost', 7860))
    sock.close()
    
    if result == 0:
        print("✅ ポート7860は使用中です（アプリが起動している可能性があります）")
    else:
        print("❌ ポート7860は使用されていません")
        print("アプリを起動するには、別のターミナルで 'make app' を実行してください")
except Exception as e:
    print(f"⚠️ ポート確認エラー: {e}")

print("\n💡 推奨: 別のターミナルで 'make app' を実行してからテストを続行してください")

---

## 🏥 2. システムヘルスチェック

システム全体の健康状態を確認し、問題があれば早期発見します。

### 🔍 基本的な接続テスト

In [ ]:
# 🏥 システム全体のヘルスチェック実行

print("🔍 システムヘルスチェックを開始します...")
print("=" * 60)

# 1. メインアプリケーションの確認
print("\n1️⃣ メインアプリケーション (http://localhost:7860)")
main_app_healthy = check_service_health("http://localhost:7860", "メイン Gradio アプリ")

# 2. API基盤の確認
print("\n2️⃣ API基盤エンドポイント")
api_endpoints = [
    ("http://localhost:7860/api/health", "ヘルスチェックAPI"),
    ("http://localhost:7860/api/system/status", "システム状態API"),
    ("http://localhost:7860/api/gradio/interfaces", "Gradioインターフェース一覧API")
]

api_healthy = []
for endpoint, name in api_endpoints:
    result = check_service_health(endpoint, name)
    api_healthy.append(result)

# 3. ファイルシステムの確認
print("\n3️⃣ 重要ファイルの存在確認")
critical_files = [
    "mysite/asgi.py",
    "app.py", 
    "routes/api.py",
    "app/Http/Controllers/Gradio/gra_01_chat/Chat.py",
    "app/Http/Controllers/Gradio/gra_11_system_monitor/system_monitor.py"
]

file_checks = []
for file_path in critical_files:
    full_path = Path(PROJECT_ROOT) / file_path
    if full_path.exists():
        print(f"✅ {file_path}: 存在")
        file_checks.append(True)
    else:
        print(f"❌ {file_path}: 存在しない")
        file_checks.append(False)

# 4. プロセス状況の確認
print("\n4️⃣ システムリソース確認")
try:
    cpu_percent = psutil.cpu_percent(interval=1)
    memory = psutil.virtual_memory()
    disk = psutil.disk_usage('/')
    
    print(f"💻 CPU使用率: {cpu_percent}%")
    print(f"🧠 メモリ使用率: {memory.percent}% ({memory.used // (1024**3)}GB / {memory.total // (1024**3)}GB)")
    print(f"💾 ディスク使用率: {disk.percent}% ({disk.used // (1024**3)}GB / {disk.total // (1024**3)}GB)")
    
    resource_healthy = cpu_percent < 80 and memory.percent < 80 and disk.percent < 90
except Exception as e:
    print(f"⚠️ リソース確認エラー: {e}")
    resource_healthy = False

# 5. 総合判定
print("\n" + "=" * 60)
print("📊 ヘルスチェック結果")
print("=" * 60)

total_checks = 1 + len(api_healthy) + len(file_checks) + 1
passed_checks = sum([main_app_healthy] + api_healthy + file_checks + [resource_healthy])

print(f"✅ 成功: {passed_checks}")
print(f"❌ 失敗: {total_checks - passed_checks}")
print(f"📈 成功率: {(passed_checks/total_checks)*100:.1f}%")

if passed_checks == total_checks:
    print("\n🎉 全てのヘルスチェックに合格しました！")
elif passed_checks >= total_checks * 0.8:
    print("\n⚠️ 一部問題がありますが、基本機能は動作しています")
else:
    print("\n🚨 重大な問題が検出されました。詳細を確認してください")

print(f"\n⏰ チェック完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## 🎯 3. 8つのGradioインターフェース個別テスト

各Gradioインターフェースの詳細テストを実行します。

### 📋 Gradioインターフェース一覧

1. **AIチャット** (`gra_01_chat`) - AI との対話機能
2. **ファイル管理** (`gra_05_files`) - ファイルアップロード・管理
3. **GitHub Issue自動生成** (`gra_03_programfromdocs`) - ドキュメントからIssue生成
4. **HTML表示** (`gra_07_html`) - HTML コンテンツの表示
5. **OpenInterpreter** (`gra_09_openinterpreter`) - コード実行・解釈
6. **記憶復元** (`gra_15_memory_restore`) - AI の記憶・復元機能
7. **GitHub Issueシステム生成** (`gra_github_issue_generator`) - システム仕様からIssue生成
8. **システム監視** (`gra_11_system_monitor`) - システムの状態監視・ヘルスチェック

In [ ]:
# 🔍 Gradioインターフェースファイルの存在確認

gradio_interfaces = [
    ("gra_01_chat", "app/Http/Controllers/Gradio/gra_01_chat/Chat.py", "AIチャット"),
    ("gra_05_files", "app/Http/Controllers/Gradio/gra_05_files/files.py", "ファイル管理"),
    ("gra_03_programfromdocs", "app/Http/Controllers/Gradio/gra_03_programfromdocs/github_issue_automation.py", "GitHub Issue自動生成"),
    ("gra_07_html", "app/Http/Controllers/Gradio/gra_07_html/gradio.py", "HTML表示"),
    ("gra_09_openinterpreter", "app/Http/Controllers/Gradio/gra_09_openinterpreter/openinterpreter.py", "OpenInterpreter"),
    ("gra_15_memory_restore", "app/Http/Controllers/Gradio/gra_15_memory_restore/memory_restore.py", "記憶復元"),
    ("gra_github_issue_generator", "app/Http/Controllers/Gradio/gra_github_issue_generator/main_interface.py", "GitHub Issueシステム生成"),
    ("gra_11_system_monitor", "app/Http/Controllers/Gradio/gra_11_system_monitor/system_monitor.py", "システム監視")
]

print("📁 Gradioインターフェースファイル確認")
print("=" * 70)

interface_status = []
for interface_id, file_path, description in gradio_interfaces:
    full_path = Path(PROJECT_ROOT) / file_path
    exists = full_path.exists()
    interface_status.append((interface_id, exists, description))
    
    if exists:
        # ファイルサイズも確認
        file_size = full_path.stat().st_size
        print(f"✅ {description} ({interface_id})")
        print(f"   📄 ファイル: {file_path}")
        print(f"   📏 サイズ: {file_size:,} bytes")
    else:
        print(f"❌ {description} ({interface_id})")
        print(f"   📄 ファイル: {file_path} - 存在しません")
    print()

# 統計表示
total_interfaces = len(interface_status)
existing_interfaces = sum(1 for _, exists, _ in interface_status if exists)

print("=" * 70)
print(f"📊 統計: {existing_interfaces}/{total_interfaces} のインターフェースが利用可能")
print(f"📈 利用可能率: {(existing_interfaces/total_interfaces)*100:.1f}%")

if existing_interfaces == total_interfaces:
    print("🎉 全てのGradioインターフェースが正常に配置されています！")
elif existing_interfaces >= total_interfaces * 0.8:
    print("⚠️ 大部分のインターフェースが利用可能です")
else:
    print("🚨 多くのインターフェースファイルが見つかりません")

---

## 🌐 4. API基盤テスト

Laravel風API基盤（`routes/api.py`）の各エンドポイントをテストします。

### 📡 API エンドポイント一覧テスト

In [ ]:
# 🌐 API基盤の全エンドポイントテスト

base_url = "http://localhost:7860"

# テスト対象のAPIエンドポイント一覧
api_endpoints_test = [
    # 基本的なヘルスチェック
    ("GET", "/api/health", "ヘルスチェック"),
    ("GET", "/api/system/status", "システム状態"),
    
    # Gradio管理API
    ("GET", "/api/gradio/interfaces", "Gradioインターフェース一覧"),
    ("GET", "/api/gradio/status", "Gradio状態"),
    
    # システム監視API
    ("GET", "/api/system/monitor/health", "システム監視・ヘルス"),
    ("GET", "/api/system/monitor/resources", "システムリソース監視"),
    ("GET", "/api/system/monitor/history", "監視履歴"),
    
    # 統計API
    ("GET", "/api/statistics/usage", "使用統計"),
    ("GET", "/api/statistics/performance", "パフォーマンス統計"),
]

print("🧪 API基盤テスト開始")
print("=" * 80)

api_test_results = []

for method, endpoint, description in api_endpoints_test:
    print(f"\n🔗 テスト: {description}")
    print(f"   {method} {endpoint}")
    
    try:
        if method == "GET":
            response = requests.get(f"{base_url}{endpoint}", timeout=10)
        elif method == "POST":
            response = requests.post(f"{base_url}{endpoint}", timeout=10)
        else:
            print(f"   ⚠️ 未対応のHTTPメソッド: {method}")
            continue
            
        print(f"   📊 ステータス: {response.status_code}")
        
        if response.status_code == 200:
            print("   ✅ 成功")
            # レスポンス内容の一部を表示（JSON形式の場合）
            try:
                data = response.json()
                if isinstance(data, dict):
                    # 重要なキーのみ表示
                    important_keys = ['status', 'message', 'data', 'success', 'timestamp']
                    summary = {k: v for k, v in data.items() if k in important_keys}
                    if summary:
                        print(f"   📄 レスポンス要約: {json.dumps(summary, indent=2, ensure_ascii=False)}")
                    else:
                        print(f"   📄 レスポンス: {str(data)[:200]}...")
                else:
                    print(f"   📄 レスポンス: {str(data)[:200]}...")
            except:
                print(f"   📄 レスポンス: {response.text[:200]}...")
            
            api_test_results.append((endpoint, True, response.status_code))
        else:
            print(f"   ❌ エラー: HTTPステータス {response.status_code}")
            print(f"   📄 エラー内容: {response.text[:200]}...")
            api_test_results.append((endpoint, False, response.status_code))
            
    except requests.exceptions.RequestException as e:
        print(f"   ❌ 接続エラー: {str(e)}")
        api_test_results.append((endpoint, False, 0))
    except Exception as e:
        print(f"   ❌ 予期しないエラー: {str(e)}")
        api_test_results.append((endpoint, False, 0))

# テスト結果の統計
print("\n" + "=" * 80)
print("📊 API基盤テスト結果")
print("=" * 80)

total_apis = len(api_test_results)
successful_apis = sum(1 for _, success, _ in api_test_results if success)

print(f"✅ 成功: {successful_apis}")
print(f"❌ 失敗: {total_apis - successful_apis}")
print(f"📈 成功率: {(successful_apis/total_apis)*100:.1f}%")

if successful_apis == total_apis:
    print("\n🎉 全てのAPIエンドポイントが正常に動作しています！")
elif successful_apis >= total_apis * 0.8:
    print("\n⚠️ 一部のAPIで問題がありますが、主要機能は動作しています")
else:
    print("\n🚨 多くのAPIで問題が発生しています。システムの確認が必要です")

# 失敗したAPIの詳細
failed_apis = [(endpoint, status_code) for endpoint, success, status_code in api_test_results if not success]
if failed_apis:
    print("\n❌ 失敗したAPI:")
    for endpoint, status_code in failed_apis:
        print(f"   • {endpoint} (ステータス: {status_code})")

print(f"\n⏰ テスト完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

---

## 🔄 5. CI/CD 自動テスト実行

プロジェクトの自動化されたテストスイートを実行します。

### 🚀 クイックテスト（GitHub Issue生成なし）

In [ ]:
# 🚀 CI/CD クイックテスト実行
print("🔄 CI/CDクイックテストを開始します...")
print("⚠️ このテストは数分かかる場合があります")
print("")

# まずは軽量なテストから実行
try:
    run_make_command("ci-quick", timeout=120)
    print("✅ CI/CDクイックテスト完了")
except Exception as e:
    print(f"❌ CI/CDクイックテストでエラーが発生: {e}")
    print("💡 手動で 'make ci-quick' を実行してみてください")

---

## 🛠️ 6. トラブルシューティング・ガイド

よくある問題と解決方法を実行可能な形で記録しています。

### ❓ 問題1: ポート7860が使用できない

In [ ]:
# 🛠️ ポート7860の問題解決

def diagnose_and_fix_port_issue():
    print("🔍 ポート7860の診断を開始します...")
    
    # 1. ポートの使用状況確認
    print("\n1️⃣ ポート使用状況の確認")
    try:
        result = subprocess.run("lsof -i:7860", shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print("📋 ポート7860を使用中のプロセス:")
            print(result.stdout)
            
            print("\n🔧 解決策: プロセスを停止します")
            run_make_command("stop-port")
        else:
            print("✅ ポート7860は使用されていません")
    except Exception as e:
        print(f"⚠️ ポート確認エラー: {e}")
    
    # 2. 再度確認
    print("\n2️⃣ 停止後の確認")
    try:
        result = subprocess.run("lsof -i:7860", shell=True, capture_output=True, text=True)
        if result.returncode == 0:
            print("⚠️ まだプロセスが残っています:")
            print(result.stdout)
            print("\n🔧 強制停止を試行します")
            subprocess.run("kill -9 $(lsof -ti:7860)", shell=True)
        else:
            print("✅ ポート7860が解放されました")
    except Exception as e:
        print(f"⚠️ 再確認エラー: {e}")
    
    # 3. 最終確認
    print("\n3️⃣ 最終確認")
    try:
        import socket
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('localhost', 7860))
        sock.close()
        
        if result == 0:
            print("⚠️ ポート7860はまだ使用中です")
        else:
            print("✅ ポート7860は利用可能です")
            print("💡 'make app' でアプリケーションを起動できます")
    except Exception as e:
        print(f"⚠️ 最終確認エラー: {e}")

# 診断・修復実行
diagnose_and_fix_port_issue()

### ❓ 問題2: データベース接続エラー（"unable to open database file"）

In [ ]:
# 🛠️ データベース接続問題の解決

def diagnose_and_fix_database_issue():
    print("🔍 データベース接続の診断を開始します...")
    
    # 1. データベースファイルの確認
    print("\n1️⃣ データベースファイルの確認")
    database_paths = [
        "db.sqlite3",
        "database/db.sqlite3", 
        "storage/database/db.sqlite3",
        "mysite/db.sqlite3"
    ]
    
    found_db = False
    for db_path in database_paths:
        full_path = Path(PROJECT_ROOT) / db_path
        if full_path.exists():
            print(f"✅ データベース発見: {db_path}")
            print(f"   📏 サイズ: {full_path.stat().st_size:,} bytes")
            print(f"   📅 更新日時: {datetime.fromtimestamp(full_path.stat().st_mtime)}")
            found_db = True
        else:
            print(f"❌ データベース無し: {db_path}")
    
    if not found_db:
        print("\n🔧 データベースファイルが見つかりません。作成します...")
        
        # 2. データベースの作成
        print("\n2️⃣ データベースの作成")
        try:
            # SQLiteデータベースの作成
            import sqlite3
            db_path = Path(PROJECT_ROOT) / "db.sqlite3"
            
            # ディレクトリが存在しない場合は作成
            db_path.parent.mkdir(parents=True, exist_ok=True)
            
            # データベース接続・作成
            conn = sqlite3.connect(str(db_path))
            cursor = conn.cursor()
            
            # 基本的なテーブル作成
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS system_logs (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
                    level TEXT NOT NULL,
                    message TEXT NOT NULL,
                    component TEXT,
                    details TEXT
                )
            ''')
            
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS gradio_sessions (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    session_id TEXT UNIQUE NOT NULL,
                    interface_name TEXT NOT NULL,
                    start_time DATETIME DEFAULT CURRENT_TIMESTAMP,
                    last_activity DATETIME DEFAULT CURRENT_TIMESTAMP,
                    status TEXT DEFAULT 'active'
                )
            ''')
            
            # 初期データの挿入
            cursor.execute('''
                INSERT INTO system_logs (level, message, component) 
                VALUES ('INFO', 'Database initialized', 'system')
            ''')
            
            conn.commit()
            conn.close()
            
            print(f"✅ データベースを作成しました: {db_path}")
            print(f"📏 サイズ: {db_path.stat().st_size:,} bytes")
            
        except Exception as e:
            print(f"❌ データベース作成エラー: {e}")
    
    # 3. 権限の確認・修正
    print("\n3️⃣ データベースファイルの権限確認")
    for db_path in database_paths:
        full_path = Path(PROJECT_ROOT) / db_path
        if full_path.exists():
            try:
                # 読み書き権限があるか確認
                test_conn = sqlite3.connect(str(full_path))
                test_conn.execute("SELECT 1")
                test_conn.close()
                print(f"✅ {db_path}: 読み書き可能")
            except Exception as e:
                print(f"❌ {db_path}: アクセスエラー - {e}")
                
                # 権限修正を試行
                try:
                    import stat
                    current_permissions = full_path.stat().st_mode
                    new_permissions = current_permissions | stat.S_IRUSR | stat.S_IWUSR
                    full_path.chmod(new_permissions)
                    print(f"🔧 {db_path}: 権限を修正しました")
                except Exception as perm_error:
                    print(f"❌ {db_path}: 権限修正失敗 - {perm_error}")
    
    print("\n✅ データベース診断・修復完了")

# 診断・修復実行
diagnose_and_fix_database_issue()

---

## 🚀 7. クイックスタート・ガイド

新しい環境や再起動後に、システムを素早く立ち上げるための手順です。

### ⚡ ワンクリック起動

In [ ]:
# 🚀 システム全体のワンクリック起動・設定

def quick_start_system():
    print("🚀 AUTOCREATEシステムのクイックスタートを開始します...")
    print("=" * 70)
    
    # ステップ1: 環境確認
    print("\n1️⃣ 環境の確認")
    print(f"📍 作業ディレクトリ: {os.getcwd()}")
    print(f"🐍 Python: {sys.version.split()[0]}")
    
    # ステップ2: 依存関係確認
    print("\n2️⃣ 依存関係の確認・インストール")
    required_packages = ['fastapi', 'gradio', 'uvicorn', 'requests', 'psutil']
    
    for package in required_packages:
        try:
            __import__(package)
            print(f"✅ {package}: インストール済み")
        except ImportError:
            print(f"❌ {package}: 未インストール")
            print(f"💡 'pip install {package}' または 'make requirements' を実行してください")
    
    # ステップ3: ポートクリア
    print("\n3️⃣ ポートのクリア")
    try:
        run_make_command("stop-port", timeout=10)
    except:
        print("⚠️ ポート停止でエラーが発生（問題ない場合があります）")
    
    # ステップ4: データベース確認
    print("\n4️⃣ データベースの確認")
    db_path = Path(PROJECT_ROOT) / "db.sqlite3"
    if db_path.exists():
        print(f"✅ データベース存在: {db_path}")
    else:
        print("❌ データベースが見つかりません")
        print("🔧 上記のデータベース修復セルを実行してください")
    
    # ステップ5: 重要ファイル確認
    print("\n5️⃣ 重要ファイルの確認")
    critical_files = [
        "mysite/asgi.py",
        "app.py",
        "routes/api.py"
    ]
    
    all_files_exist = True
    for file_path in critical_files:
        full_path = Path(PROJECT_ROOT) / file_path
        if full_path.exists():
            print(f"✅ {file_path}")
        else:
            print(f"❌ {file_path}: 見つかりません")
            all_files_exist = False
    
    # ステップ6: 起動準備
    print("\n6️⃣ 起動準備の完了判定")
    if all_files_exist:
        print("✅ 全ての必要ファイルが揃っています")
        print("\n🚀 システム起動の準備完了！")
        print("\n次のステップ:")
        print("1. 別のターミナルで 'make app' を実行")
        print("2. ブラウザで http://localhost:7860 にアクセス")
        print("3. このノートブックの「システムヘルスチェック」を実行して動作確認")
        
        # 自動でブラウザを開く提案
        print("\n💡 このセルの実行完了後、以下のリンクをクリックしてアクセス:")
        print("🔗 http://localhost:7860")
        
    else:
        print("❌ 一部のファイルが不足しています")
        print("🔧 プロジェクトの復元または再構築が必要です")
    
    print(f"\n⏰ クイックスタート完了: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# クイックスタート実行
quick_start_system()

---

## 📚 8. このノートブックの活用・更新ガイド

### 🔄 定期的なメンテナンス

このノートブックを最新状態に保つために：

1. **新しい機能追加時**: 対応するテストセルを追加
2. **エラー発生時**: トラブルシューティングセクションに解決方法を記録
3. **設定変更時**: クイックスタートガイドを更新
4. **APIエンドポイント追加時**: API基盤テストセクションを更新

### 📖 使い方のコツ

- **セルの実行順序**: 上から順番に実行することを推奨
- **エラー対応**: エラーが発生したセルは、トラブルシューティングセクションを参照
- **定期実行**: システム稼働中は定期的にヘルスチェックを実行
- **ログ記録**: 実行結果は重要な場合は外部ファイルにも保存

### 🤝 チーム・AI協働での活用

- **新メンバー**: このノートブックでシステム全体を理解
- **AI引き継ぎ**: 次のAIはこのノートブックで現状把握
- **問題共有**: エラー発生時は該当セルの実行結果を共有
- **改善提案**: 新しいテスト方法や改善案をセルとして追加

---

## 🎯 まとめ

この動くドキュメント（Jupyter Notebook）により：

✅ **テストの統一化** - 全ての機能テストが一箇所に集約  
✅ **忘れ防止** - Makeコマンドやトラブルシューティング手順を記録  
✅ **即座の実行** - 説明を読みながら実際にテストを実行  
✅ **継続的改善** - 新しい問題や解決方法を随時追加  
✅ **知識継承** - 人間・AI問わず、誰でもシステムを理解・操作可能  

**次のステップ**: このノートブックを定期的に実行し、システムの健全性を維持してください。

---

*📅 最終更新: 2025年6月15日*  
*🤖 作成者: GitHub Copilot + Human 協働*